# Få tak i værdata

In [2]:
from openai import OpenAI
import json

client = OpenAI()

messages = [
  {
    "role": "system",
    "content": "You are a weather reporter.",
  },
  {
    "role": "user",
    #"content": "Hvordan er været i Oslo?",
    "content": "Hvordan er været i Oslo? Og hva med Trondheim? Og hva er temperaturforskjellen mellom de to byene?",
  }
]

functions = [
  {
    "name": "get_weather",
    "description": "Get the weather for a given location.",
    "parameters": {
      "type": "object",
      "properties": {
        "location": {
          "type": "string",
          "description": "The location to get the weather for.",
        }
      },
      "required": ["location"],
    }
  }
]

def get_weather(location):
  import requests
  response = requests.get(f"https://wttr.in/{location}?format=j1").json()
  return json.dumps(response["current_condition"][0])

function_calls = {
  "get_weather": lambda args: get_weather(args["location"])
}

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
)

i = 0
while True:
  if i > 10:
    print("Too many function calls.")
    break
  i += 1

  if completion.choices[0].finish_reason == "stop":
    print()
    print("Response:")
    print(completion.choices[0].message.content)
    break

  if completion.choices[0].finish_reason == "function_call":
    function_name = completion.choices[0].message.function_call.name
    function_args = completion.choices[0].message.function_call.arguments
    print(f"Calling function: \"{completion.choices[0].message.function_call.name}\" with parameters: " + str(function_args).replace("\n", " ").replace("   ", " "))
    value = function_calls[function_name](json.loads(function_args))
    print("Sending function return value back to GPT")
    print()

    messages.append({
        "role": "assistant",
        "content": None,
        "function_call": {
            "name": function_name,
            "arguments": function_args,
        },
    })

    messages.append({
        "role": "function",
        "content": str(value),
        "name": function_name,
    })

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        temperature=0,
    )


Calling function: "get_weather" with parameters: { "location": "Oslo" }
Sending function return value back to GPT

Calling function: "get_weather" with parameters: { "location": "Trondheim" }
Sending function return value back to GPT


Response:
Været i Oslo er delvis skyet med en temperatur på 7°C (45°F). Det er en skydekke på 75% og en luftfuktighet på 93%. Vinden blåser fra sør med en hastighet på 61 km/t (38 mph). Siktbarheten er god med 10 km (6 miles).

I Trondheim er det solrikt med en temperatur på -15°C (5°F). Det er ingen skydekke og en luftfuktighet på 85%. Vinden blåser fra øst-sørøst med en hastighet på 11 km/t (7 mph). Siktbarheten er også god med 10 km (6 miles).

Temperaturforskjellen mellom Oslo og Trondheim er 22°C (40°F), med Oslo som er varmere.
